<a href="https://colab.research.google.com/github/JacekSkrzeszewski/PSI24/blob/main/Prolog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Zadanie 1: Drzewo genealogiczne

In [ ]:
% Fakty
ojciec(jan, anna).
ojciec(jan, piotr).
ojciec(zygmunt, jan).
ojciec(zygmunt, andrzej).
ojciec(andrzej, tomasz).
matka(maria, anna).
matka(maria, piotr).
malzenstwo(jan, maria).
mezczyzna(jan).
mezczyzna(piotr).
mezczyzna(zygmunt).
mezczyzna(andrzej).
mezczyzna(tomasz).
kobieta(maria).
kobieta(anna).

% Reguly
rodzic(X,Y) :- ojciec(X,Y); matka(X,Y).
przodek(X,Y) :- rodzic(X,Y); rodzic(X,Z), przodek(Z,Y).
brat(X,Y) :- rodzic(Z,Y), rodzic(Z,X), X \= Y, mezczyzna(X), !.
siostra(X,Y) :- rodzic(Z,Y), rodzic(Z,X), X \= Y, kobieta(X), !.
kuzyn(X,Y) :- rodzic(Z,C), rodzic(Z,V), rodzic(C,X), rodzic(V,Y), C \= V, mezczyzna(X).

% Zapytania
?- przodek(jan, anna).
?- brat(piotr, anna).
?- siostra(anna,piotr).
?- kuzyn(X, piotr).

Zadanie 2: System rezerwacji lotów

In [ ]:
% Fakty
lotnisko(warszawa).
lotnisko(krakow).
lotnisko(gdansk).

polaczenie(warszawa, krakow, 300).
polaczenie(krakow, gdansk, 200).
polaczenie(warszawa, gdansk, 500).

% Reguły
polaczenie(X, Y) :- polaczenie(X, Y, _).

koszt_podrozy(X, Y, C) :-
    polaczenie(X, Y, C).

najkrotsza_droga(X, Y, Trasa, Koszt) :-
    findall([Trasa, Koszt], trasa(X, Y, [X], Trasa, Koszt), MozliweTrasy),
    sort(2, @=<, MozliweTrasy, PosortowaneTrasy),
    PosortowaneTrasy = [[Trasa, Koszt]|_].
trasa(X, X, Sciezka, Sciezka, 0).
trasa(X, Y, SciezkaDotychczasowa, Sciezka, Koszt) :-
    polaczenie(X, Z, C),
    \+ member(Z, SciezkaDotychczasowa),
    trasa(Z, Y, [Z|SciezkaDotychczasowa], Sciezka, KosztPozostaly),
    Koszt is C + KosztPozostaly.

% Zapytania
?- polaczenie(warszawa, krakow).
?- koszt_podrozy(warszawa, gdansk, C).
?- najkrotsza_droga(warszawa, gdansk, TrasaPrzez, Koszt).


Zadanie 3: System zarządzania magazynem

In [ ]:
% Fakty
:- dynamic produkt/3.
produkt(tv, 10, sektor_a).
produkt(pralka, 5, sektor_b).
produkt(lodowka, 3, sektor_c).

% Reguły
dostepny_produkt(X) :-
    produkt(X, Ilosc, _),
    Ilosc > 0.

przenies_produkt(X, Y) :-
    produkt(X, Ilosc, StarySektor),
    retract(produkt(X, Ilosc, StarySektor)),   assert(produkt(X, Ilosc, Y)),
    format('produkt(~w, ~w, ~w).~n', [X, Ilosc, Y]).

uzupelnij_stan(X, N) :-
    produkt(X, Ilosc, Sektor),
    retract(produkt(X, Ilosc, Sektor)),
    NowaIlosc is Ilosc + N,
    assert(produkt(X, NowaIlosc, Sektor)),
    format('produkt(~w, ~w, ~w).~n', [X, NowaIlosc, Sektor]).

% Zapytania
?- dostepny_produkt(tv).
?- przenies_produkt(tv, sektor_b).
?- uzupelnij_stan(pralka, 3).

Zadanie 4: System planowania trasy kuriera

In [ ]:
% Fakty
miasto(warszawa).
miasto(krakow).
miasto(poznan).
droga(warszawa, krakow, 3).
droga(krakow, poznan, 3).
droga(warszawa, poznan, 7).

% Reguły
droga(X,Y) :- droga(X,Y,_).
czas_przejazdu(X,Y,T) :- droga(X,Y,T).

najkrotsza_trasa(X, Y, NajkrotszaTrasa, NajkrotszyCzas) :-
    findall((Trasa, Czas), trasa(X, Y, Trasa, Czas), MozliweTrasy),
    minimalna_trasa(MozliweTrasy, (NajkrotszaTrasa, NajkrotszyCzas)).

trasa(X, Y, [X, Y], Czas) :- droga(X, Y, Czas).
trasa(X, Y, [X | ResztaTrasy], Czas) :-
    droga(X, Z, Czas1),
    trasa(Z, Y, ResztaTrasy, Czas2),
    X \= Y,
    \+ member(X, ResztaTrasy),
    Czas is Czas1 + Czas2.
minimalna_trasa([Trasa], Trasa).
minimalna_trasa([(T1, C1), (_, C2) | Reszta], Min) :-
    C1 =< C2, minimalna_trasa([(T1, C1) | Reszta], Min).
minimalna_trasa([(_, C1), (T2, C2) | Reszta], Min) :-
    C1 > C2, minimalna_trasa([(T2, C2) | Reszta], Min).

% Zapytania
?- droga(warszawa, krakow).
?- czas_przejazdu(warszawa, poznan, T).
?- najkrotsza_trasa(warszawa, poznan, Trasa_przez, Czas).

Zadanie 5: System rekomendacji książek

In [ ]:
% Fakty
ksiazka('Tajemnica Watsona', smith, przygodowa).
ksiazka('Smocze Wedrowki', green, science_fiction).
ksiazka('Milosc i Nienawisc', brown, dramat).
ksiazka('Krolestwo Cieni', black, horror).
ksiazka('Magiczne Zaklecia', white, kryminal).
ksiazka('Sprawiedliwosc i Zemsta', red, thriller).
ksiazka('W pustyni i w puszczy', sienkiewicz, przygodowa).
ksiazka('Hobbit', tolkien, fantasy).
ksiazka('Pan Tadeusz', mickiewicz, poezja).

:- dynamic ocena/3.

ocena(uzytkownik1, 'Tajemnica Watsona', 4).
ocena(uzytkownik1, 'Smocze Wedrowki', 5).
ocena(uzytkownik1, 'Milosc i Nienawisc', 3).
ocena(uzytkownik1, 'Krolestwo Cieni', 4).
ocena(uzytkownik1, 'Magiczne Zaklecia', 5).
ocena(uzytkownik1, 'Sprawiedliwosc i Zemsta', 4).
ocena(uzytkownik1, 'W pustyni i w puszczy', 5).
ocena(uzytkownik1, 'Hobbit', 5).
ocena(uzytkownik1, 'Pan Tadeusz', 3).
ocena(uzytkownik2, 'Tajemnica Watsona', 5).
ocena(uzytkownik2, 'Smocze Wedrowki', 4).
ocena(uzytkownik2, 'Milosc i Nienawisc', 4).
ocena(uzytkownik2, 'Krolestwo Cieni', 5).
ocena(uzytkownik2, 'Magiczne Zaklecia', 3).
ocena(uzytkownik2, 'Sprawiedliwosc i Zemsta', 2).
ocena(uzytkownik2, 'W pustyni i w puszczy', 4).
ocena(uzytkownik2, 'Hobbit', 4).
ocena(uzytkownik2, 'Pan Tadeusz', 5).

preferencje(uzytkownik1, fantasy).
preferencje(uzytkownik1, thriller).
preferencje(uzytkownik1, poezja).
preferencje(uzytkownik2, kryminal).
preferencje(uzytkownik2, romans).
preferencje(uzytkownik2, science_fiction).

% Reguły

dodaj_ocene(Uzytkownik, Ksiazka, Ocena) :-
    retractall(ocena(Uzytkownik, Ksiazka, _)),
    assertz(ocena(Uzytkownik, Ksiazka, Ocena)),
    write('Ocena zaktualizowana.'), nl.

polecane_ksiazki(Uzytkownik, Gatunek) :-
    preferencje(Uzytkownik, Gatunek),
    findall(Ksiazka,
            (ocena(Uzytkownik, Ksiazka, Ocena),
             ksiazka(Ksiazka, _, Gatunek),
             Ocena >= 4),
            ListaKsiazek),
    (ListaKsiazek = [] ->
        write('Brak ksiazek do polecenia'), nl;
    format('Polecane ksiazki: ~w', [ListaKsiazek]), nl).

% Zapytania
?- polecane_ksiazki(uzytkownik1, fantasy).
?- dodaj_ocene(uzytkownik1, 'Hobbit', 3).